In [ ]:
# dependencies import
from common_dependencies import *
from typing import NamedTuple
#import model_versions as mv
import logging
logger = logging.getLogger(f'main.data_research')

In [ ]:
MAX_VAL = 1000 # не трогать
XSHIFT = 200 # не трогать
dataset_desc = {'run_1': (DatasetPartDescription(PATH_TO_DATA['run_1'],DataCrop(0,0,MAX_VAL,MAX_VAL),SlidingCrop(1,1),XSHIFT),),
                'run_2': (DatasetPartDescription(PATH_TO_DATA['run_2'],DataCrop(0,0,MAX_VAL,MAX_VAL),SlidingCrop(1,1),XSHIFT),)}

In [ ]:
#входные и выходные данные
# reading
dataset = {'run_1':dict(zip(['x','y','bin'], [np.array(list(gen)) for gen in chain_dataset_gens(dataset_desc['run_1'])])),
           'run_2':dict(zip(['x','y','bin'], [np.array(list(gen)) for gen in chain_dataset_gens(dataset_desc['run_2'])]))}

# squueze datasets
for dataset_part_name, dataset_part in dataset.items():
    for data_part_name, data_part in dataset_part.items():
        if data_part_name == 'x':
            dataset[dataset_part_name][data_part_name] = data_part.reshape(-1,64)
        if data_part_name == 'y':
            dataset[dataset_part_name][data_part_name] = data_part.reshape(-1)
        if data_part_name == 'bin':
            dataset[dataset_part_name][data_part_name] = data_part.reshape(-1)

In [ ]:
'''def add_constant_index_level(df: pd.DataFrame, value, level_name: str):
    """Add a new level to an existing index where every row has the same, given value.
    
    Args:
        df: Any existing pd.DataFrame.
        value: Value to be placed in every row of the new index level.
        level_name: Title of the new index level.
    
    Returns:
        df with an additional, prepended index level.
    """
    return pd.concat([df], keys=[value], names=[level_name])

def unpad(arr):
    for i, t in enumerate(arr):
        if t != 0:
            return arr[i:]
    return None

def reshape_data(x_df, y_df, run):
    x_df = x_df.stack(0, future_stack=True)
    y_df = y_df.stack(0, future_stack=True)
    df = pd.concat([x_df, y_df], axis=1)
    df = df.rename(columns={0: "data", 1: "defect_state"})
    df.index = df.index.set_names('detector', level=1)
    df.index = df.index.map(lambda x: (int(x[0]), int(x[1].split('_')[-1])))
    df = add_constant_index_level(df,run,'run')
    df['time'] = df.apply(lambda x: unpad(x['data'][:32]), axis=1)
    df['data'] = df.apply(lambda x: unpad(x['data'][32:]), axis=1)
    df = df.rename(columns={'data':'amp'})
    return df

df = pd.concat([reshape_data(*dataset.get_x_and_y_data_dfs(dw.DataPart(path_to_run_folder=PATH_TO_DATA['run_1'])),1),
                reshape_data(*dataset.get_x_and_y_data_dfs(dw.DataPart(path_to_run_folder=PATH_TO_DATA['run_2'])),2)],axis=0)
df = df[['time','amp','defect_state']]
df = df.dropna()
df['first_time'] = df.apply(lambda x: x['time'][0], axis=1)
df['last_time'] = df.apply(lambda x: x['time'][-1], axis=1)
df['duration'] = df.apply(lambda x: x['last_time']-x['first_time'], axis=1)
df['dots_count'] = df.apply(lambda x: x['time'].size, axis=1)
#df = df.unstack('run')
display(df)

df_1 = df.xs(1, level='run', drop_level=False)
display(df_1)

df_1[['first_time', 'last_time', 'duration', 'dots_count']].hist()''';

In [ ]:
def unpad(arr):
    for i, t in enumerate(arr):
        if t != 0:
            return arr[i:]
    return None

In [ ]:
'''for dataset_part_name, dataset_part in dataset.items():
    for i, value in enumerate(dataset_part['x']):
        time = value[0:32]
        amp = value[32:]
        
        print('|'*8,'step','|'*8)
        print('|'*8, 'time')
        print('orig', time)
        print('normalize time', dw.normalize_data(time))
        print('standardize time', dw.standardize_data(time))
        print('|'*8, 'amp')
        print('orig', amp)
        print('normalize amp', dw.normalize_data(amp))
        print('standardize amp', dw.standardize_data(amp))
        #print('unpad time: ', unpad(time))
        #print('unpad amp: ', unpad(amp))
        print()
        if i == 2:
            break
    break''';

"""plt.figure(figsize=(12, 6))
ax1 = plt.subplot(2,3,1)
ax2 = plt.subplot(2,3,2)
ax3 = plt.subplot(2,3,3)
ax4 = plt.subplot(2,1,2)
axes = [ax1, ax2, ax3, ax4]""";

In [ ]:
FONT_SIZE = 16
fig, axes = plt.subplots(4,5, gridspec_kw={'width_ratios': [10, 1, 1, 1, 1]}) # [10, 1, 1, 1, 1] [6, 1, 1, 1, 1]
fig.set_figwidth(12*5) # 12*5 8*5
fig.set_figheight(8*4)

BINS = 20
ALPHA = 0.5

axes_it = iter(axes)
def_names = ['Defect','Non defect']
for dataset_part_name, dataset_part in dataset.items():
    for def_n in def_names:
        values = dataset_part['x']
        if def_n == 'Defect':
            values = values[dataset_part['bin']>0]
            col = '#ff243d'
        else:
            values = values[dataset_part['bin']==0]
            col = '#24a0ff'

        temp_ax = next(axes_it)
        times = [unpad(value) for value in values[:,:32] if np.count_nonzero(value)]
        amps = [unpad(value) for value in values[:,32:] if np.count_nonzero(value)]

        # 1 график \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
        
        temp_ax[0].set_title(f'{def_n} graphs for {dataset_part_name} (Total={values.shape[0]})',fontsize=FONT_SIZE)
        for time, amp in zip([time for time in times if time.size == 31], 
                             [amp for amp in amps if amp.size == 31]):
            temp_ax[0].plot(time, amp, c=col)
        for time, amp in zip([time for time in times if time.size != 31], 
                             [amp for amp in amps if amp.size != 31]):
            temp_ax[0].plot(time, amp, c='yellow')
        temp_ax[0].set_ylabel('Amplitude',fontsize=FONT_SIZE)

        # 2 график \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
        
        temp_ax[1].set_title(f'First time value',fontsize=FONT_SIZE)
        quantities, bins, patches = temp_ax[1].hist([time[0] for time in times], bins=BINS, alpha=ALPHA, color=col)
        table = temp_ax[1].table(cellText=[[bound, str(quant)] for bound, quant in 
                      zip([f'{bins[i]:.2f} to {bins[i+1]:.2f}' for i in range(len(bins)-1)], quantities)],
                      colLabels=['num','boundaries','plots count'], loc='upper center', fontsize=FONT_SIZE)
        table.set_fontsize(FONT_SIZE)
        table.scale(1,1.5)

        for i, bound in enumerate(bins):
            temp_ax[0].axvline(x = bound, color = 'black', ymax = 0.4)
            if i < len(bins)-1:
                temp_ax[0].annotate(text=f'{i}', xy=[bound, -40], color='black', fontsize=FONT_SIZE) # ha='center', va='center'
            

        # 3 график \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
        
        temp_ax[2].set_title(f'Last time value',fontsize=FONT_SIZE)
        quantities, bins, patches = temp_ax[2].hist([time[-1] for time in times], bins=BINS, alpha=ALPHA, color=col)
        table = temp_ax[2].table(cellText=[[bound, str(quant)] for bound, quant in 
                              zip([f'{bins[i]:.2f} to {bins[i+1]:.2f}' for i in range(len(bins)-1)], quantities)],
                              colLabels=['num','boundaries','plots count'], loc='upper center', fontsize=FONT_SIZE)
        table.set_fontsize(FONT_SIZE)
        table.scale(1,1.5)

        for i, bound in enumerate(bins):
            temp_ax[0].axvline(x = bound, color = 'black', ymin = 0.6)
            if i < len(bins)-1:
                temp_ax[0].annotate(text=f'{i}', xy=[bound, 40], color='black', fontsize=FONT_SIZE) # ha='center', va='center'
        # 4 график \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
        
        temp_ax[3].set_title(f'Duration',fontsize=FONT_SIZE)
        quantities, bins, patches = temp_ax[3].hist([time[-1]-time[0] for time in times], bins=BINS, alpha=ALPHA, color=col)
        table = temp_ax[3].table(cellText=[[bound, str(quant)] for bound, quant in 
                                   zip([f'{bins[i]:.2f} to {bins[i+1]:.2f}' for i in range(len(bins)-1)], quantities)],
                                   colLabels=['num','boundaries','plots count'], loc='upper center', fontsize=FONT_SIZE)
        table.set_fontsize(FONT_SIZE)
        table.scale(1,1.5)

        temp_ax[4].set_title(f'Points counts',fontsize=FONT_SIZE)
        values, quantities =np.unique(np.array([time.size for time in times]), return_counts=True)
        table = temp_ax[4].table(cellText=[[str(val), str(quant)] for val, quant in zip(values, quantities)],
                                   colLabels=['points count','plots count'], loc='upper center', fontsize=FONT_SIZE)
        table.set_fontsize(FONT_SIZE)
        table.scale(1,1.5)

for row_ax in axes:
    for i, ax in enumerate(row_ax):
        ax.grid(True)
        ax.tick_params(axis='both', labelsize=FONT_SIZE)
        if i != 4:
            ax.set_xlabel('Time',fontsize=FONT_SIZE)
        else:
            ax.set_xlabel('Dots count',fontsize=FONT_SIZE)
        if i == 0:
            ax.set_xlim(14,45)
            ax.xaxis.set_major_locator(ticker.MultipleLocator(1)) 
            ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.d')) 
        '''if i:
            ax.set_ylabel('Plots quantity',fontsize=FONT_SIZE)
            ax.xaxis.tick_top()
            ax.xaxis.set_label_position('top')'''
#  Устанавливаем интервал основных делений: 
#ax[1].xaxis.set_major_locator(ticker.MultipleLocator(1)) 
#  Устанавливаем форматирование чисел основных делений: 
#ax[1].xaxis.set_major_formatter(ticker.FormatStrFormatter('%.d')) 
#  Устанавливаем форматирование делений: 
#ax[1].xaxis.set_tick_params(which = 'major', labelsize = FONT_SIZE, labelrotation = 90) 
    
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.1, hspace=0.2)
plt.show()

In [ ]:
'''plt.plot(np.arange(10),np.arange(10)**2)
plt.xticks(list(plt.xticks()[0]))
plt.xticks([1.1,2.4,6.2], minor=True)
plt.show()''';

In [ ]:
'''from matplotlib.transforms import Bbox
FONT_SIZE = 16
fig, ax = plt.subplots()
fig.set_figwidth(10)
fig.set_figheight(6)

quantities, bins, patches = ax.hist(np.arange(20))
print(quantities,len(quantities))
print(bins,len(bins))
print([f'{bins[i]:.2f} to {bins[i+1]:.2f}' for i in range(len(bins)-1)])
columns = ['quantity']
table = ax.table(cellText=[[bound, str(quant)] for bound,quant in zip([f'{bins[i]:.2f} to {bins[i+1]:.2f}' for i in range(len(bins)-1)], quantities)],
                      colLabels=['boundaries','quantity'])
table.set_fontsize(FONT_SIZE)
table.scale(1.5, 1.5)
#table.scale(1, 4)
ax.xaxis.tick_top()
ax.xaxis.set_label_position('top')
plt.subplots_adjust(left=0.2, bottom=0.2)
plt.show()''';